**Instituto de Informática - UFG**

Disciplina: Inteligência Artificial

Prof. Cedric Luiz de Carvalho


Grupo (Gabriel Olímpio Rocha Leão, Geovanny Magalhães Novais)

---


Analista de dados: Gabriel Olímpio Rocha Leão

---

Engenheiro(a) do conhecimento: Geovanny Magalhães Novais

---


#Redes Neurais





Utilizando a base de dados [Patient Survival Prediction](https://www.kaggle.com/mitishaagarwal/patient?select=dataset.csv), para verificar estados de pacientes com risco de morte, após processamento prévio feito na etapa de análise de dados, obtivemos um novo dataset que será explorado agora: 


* Montando o drive para acessar a database 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


* Criando o dataframe

In [ ]:
import pandas as pd
dados = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Trabalho3/databaseDadosValidos.csv')

* Exibindo valores de exemplo

In [ ]:
dados.head()

,Unnamed: 0,age,bmi,elective_surgery,ethnicity,gender,height,icu_admit_source,icu_stay_type,icu_type,...,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,apache_3j_bodysystem,apache_2_bodysystem,hospital_death
0,0,68.0,22.730000,0,Caucasian,M,180.3,Floor,admit,CTICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Sepsis,Cardiovascular,0
1,1,77.0,27.420000,0,Caucasian,F,160.0,Floor,admit,Med-Surg ICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0
2,5,67.0,27.560000,0,Caucasian,M,190.5,Accident & Emergency,admit,Med-Surg ICU,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,Neurological,Neurologic,0
3,10,72.0,28.257052,1,Hispanic,F,154.9,Operating Room / Recovery,admit,Med-Surg ICU,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Respiratory,Respiratory,0
4,17,46.0,25.845717,0,Hispanic,M,167.6,Accident & Emergency,admit,CTICU,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Respiratory,Respiratory,0


* Criando um dataframe somente com colunas numéricas.

Visto a dificuldade em converter cada uma das colunas em colunas numéricas, independentes entre si, para que possam ser usadas pela Rede Neural.

In [ ]:
dados.drop('ethnicity', axis=1, inplace=True)
dados.drop('gender', axis=1, inplace=True)
dados.drop('icu_admit_source', axis=1, inplace=True)
dados.drop('icu_stay_type', axis=1, inplace=True)
dados.drop('icu_type', axis=1, inplace=True)
dados.drop('apache_3j_bodysystem', axis=1, inplace=True)
dados.drop('apache_2_bodysystem', axis=1, inplace=True)

In [ ]:
dados.head()

,Unnamed: 0,age,bmi,elective_surgery,height,pre_icu_los_days,weight,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,...,apache_4a_icu_death_prob,aids,cirrhosis,diabetes_mellitus,hepatic_failure,immunosuppression,leukemia,lymphoma,solid_tumor_with_metastasis,hospital_death
0,0,68.0,22.730000,0,180.3,0.541667,73.9,113.0,502.01,0,...,0.05,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,1,77.0,27.420000,0,160.0,0.927778,70.2,108.0,203.01,0,...,0.29,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,5,67.0,27.560000,0,190.5,0.000694,100.0,301.0,403.01,0,...,0.02,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
3,10,72.0,28.257052,1,154.9,0.004861,67.8,303.0,1304.08,1,...,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
4,17,46.0,25.845717,0,167.6,0.000000,72.6,108.0,203.01,0,...,0.06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


* Definindo variáveis dependentes e independentes

In [ ]:
x = dados.iloc[:,1:74] 
y = dados.iloc[:,:74]

* Garantindo que o sistema só lidará com dados inteiros, com a biblioteca numpy
* Mostrando que tamanho dos dados é o mesmo que o de x

In [ ]:
import numpy as np
x = np.asarray(x).astype('float32')
y = np.asarray(y).astype('float32')
len(dados) - len(x)

0

* Importando o modelo de treino sklearn
* Treinando a RNA com proporção de treino 80% e 20% de teste
* Criando números aleatórios com RandomState para aumentar a aleatoriedade dos treinos

In [ ]:
from sklearn.model_selection import train_test_split
rng = np.random.RandomState(0)
xDeTreino, xDeTeste, yDeTreino, yDeTeste = train_test_split(x, y, test_size = 0.2, random_state = rng)

* Importando a biblioteca keras, além de seus modelos sequenciais e de suas camadas densas

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense

### Decidimos no projeto nos ater ao modelo de ativação relu

* Adicionando uma RNA de três camadas com iniciação uniforme
* As camadas 1 e 2 tem 10 neurônios, enquanto a 3 possui somente um

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 10, kernel_initializer='uniform', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 10, kernel_initializer='uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='uniform', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 5)

Epoch 1/5
4495/4495 [==============================] - 9s 2ms/step - loss: -10762.1523 - accuracy: 0.0222
Epoch 2/5
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.2021 - accuracy: 0.0219
Epoch 3/5
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.2021 - accuracy: 0.0219
Epoch 4/5
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.1943 - accuracy: 0.0219
Epoch 5/5
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.1973 - accuracy: 0.0219


* Como obtivemos um valor baixíssimo de previsão, tentamos modificar a rede, modificando um dos componentes da rede e e mantendo os demais componentes sem alteração 

* Primeiro, tentamos aumentar o número de neurônios:

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 100, kernel_initializer='uniform', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 50, kernel_initializer='uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='uniform', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 5)

Epoch 1/5
4495/4495 [==============================] - 10s 2ms/step - loss: -10793.7305 - accuracy: 0.0220
Epoch 2/5
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.2012 - accuracy: 0.0219
Epoch 3/5
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.1914 - accuracy: 0.0219
Epoch 4/5
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.1934 - accuracy: 0.0219
Epoch 5/5
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.1816 - accuracy: 0.0219


 Ao multiplicarmos por dez o número de neurônios na camada 1 e por cinco na camada 2, não obtivemos melhora

* Tentamos então, aumentar o número de epócas:

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 10, kernel_initializer='uniform', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 10, kernel_initializer='uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='uniform', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 10)

Epoch 1/10
4495/4495 [==============================] - 9s 2ms/step - loss: -10772.1885 - accuracy: 0.0222
Epoch 2/10
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.2041 - accuracy: 0.0219
Epoch 3/10
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.1885 - accuracy: 0.0219
Epoch 4/10
4495/4495 [==============================] - 13s 3ms/step - loss: -10800.2041 - accuracy: 0.0219
Epoch 5/10
4495/4495 [==============================] - 12s 3ms/step - loss: -10800.1982 - accuracy: 0.0219
Epoch 6/10
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.2002 - accuracy: 0.0219
Epoch 7/10
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.2139 - accuracy: 0.0219
Epoch 8/10
4495/4495 [==============================] - 8s 2ms/step - loss: -10800.2119 - accuracy: 0.0219
Epoch 9/10
4495/4495 [==============================] - 9s 2ms/step - loss: -10800.2012 - accuracy: 0.0219
Epoch 10/10
4495/4495 [============

Notamos que o problema se manteve

* Após o feito, tentamos então, mudar o parâmetro de inicialização do kernel:

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 10, kernel_initializer='normal', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 10, kernel_initializer='normal', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='normal', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 5)

Epoch 1/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0742 - accuracy: 0.1886
Epoch 2/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0488 - accuracy: 0.1886
Epoch 3/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0664 - accuracy: 0.1886
Epoch 4/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0645 - accuracy: 0.1886
Epoch 5/5
4495/4495 [==============================] - 12s 3ms/step - loss: 10940.0615 - accuracy: 0.1886


Conseguimos então, uma melhora relevante na taxa de acertos e tentamos outras combinações

* Usando inicializador normal na camada 1 e inicializador uniform na camada 2 e 3:

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 10, kernel_initializer='normal', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 10, kernel_initializer='uniform', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='uniform', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 5)

Epoch 1/5
4495/4495 [==============================] - 10s 2ms/step - loss: 10940.0596 - accuracy: 0.1886
Epoch 2/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0869 - accuracy: 0.1886
Epoch 3/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0908 - accuracy: 0.1886
Epoch 4/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0791 - accuracy: 0.1886
Epoch 5/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0674 - accuracy: 0.1886


Notamos que a acurácia se manteve, porém o tempo para chegar até ela diminiu

* Tentamos por fim, criar mais uma camada densa com inicialização normal:

In [ ]:
classifier = Sequential()

classifier.add(Dense(units = 10, kernel_initializer= 'normal', activation = 'relu', input_dim = 73))

classifier.add(Dense(units = 10, kernel_initializer='normal', activation = 'relu'))

classifier.add(Dense(units = 10, kernel_initializer='normal', activation = 'relu'))

classifier.add(Dense(units = 1, kernel_initializer='normal', activation='relu'))

classifier.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])

classifier.fit(xDeTreino, yDeTreino, batch_size =10, epochs = 5)

Epoch 1/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0801 - accuracy: 0.1886
Epoch 2/5
4495/4495 [==============================] - 9s 2ms/step - loss: 10940.0645 - accuracy: 0.1886
Epoch 3/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0762 - accuracy: 0.1886
Epoch 4/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0693 - accuracy: 0.1886
Epoch 5/5
4495/4495 [==============================] - 8s 2ms/step - loss: 10940.0771 - accuracy: 0.1886


A acurácia se manteve igual as anteriores, mesmo com o investimento de processamento a mais

# Conclusão

* A melhor solução custo-benefício foi a penúltima, com com uma camada normal de 10 neurônios, e duas uniformes (a primeira com 10 neurônios e a última com um), além do batch com tamanho 10, e a execução de 5 épocas. Por possuir a mesma acurácia das melhores encontradas e uma maior velocidade que as demais.

* A predição de um caso de vida ou morte é uma tarefa complexa, que exigiria um maior domínio sobre a área alvo, o que ressalta a  necessidade de uma quantidade relevante de estudo teórico e testes práticos para construir soluções sólidas, para o mercado de IA.
Desta forma, a resolução acima é o efeito de sucessivas tentativas de fazê-lo, da melhor maneira conhecida pelos autores, no momento da criação deste, não sendo viável na prática.